# 2.2 Integration with pandas data frames

As was shown earlier, we can use indexes to search for specific data points.
One way to operate on the data is using pandas data frames.

*Please note: The following steps require the pandas package.*

In [ ]:
import pandas as pd
import signac

project = signac.get_project(root="projects/tutorial")

Let's first create a basic index and use it to construct an index data frame:

In [ ]:
df_index = pd.DataFrame(project.index())
df_index.head()

It is a good idea, to explicitly use the `_id` value as index key:

In [ ]:
df_index = df_index.set_index(["_id"])
df_index.head()

Furthermore, the index would be more useful if each statepoint parameter had its own column.

In [ ]:
statepoints = {doc["_id"]: doc["statepoint"] for doc in project.index()}
df = pd.DataFrame(statepoints).T.join(df_index)
df.head()

Now we can select specific data subsets, for example to calculate the mean gas volume of argon for a pressure *p* between 2.0 and 5.0:

In [ ]:
df[(df.fluid == "argon") & (df.p > 2.0) & (df.p <= 5.0)].V_gas.mean()

Or we can plot a p-V phase diagram for argon (*requires matplotlib*).

In [ ]:
%matplotlib inline

df_water = df[df.fluid == "argon"][["p", "V_liq", "V_gas"]]
df_water.sort_values("p").set_index("p").plot(logy=True)

Or we group the data by fluid and compare the gas densities for low pressures:

In [ ]:
from matplotlib import pyplot as plt

for fluid, group in df[df.p < 2].groupby("fluid"):
    d = group.sort_values("p")
    plt.plot(d["p"], d["V_gas"] / d["N"], label=fluid)
plt.xlabel("p")
plt.ylabel(r"$\rho_{gas}$")
plt.legend(loc=0)
plt.show()